## This is an example using h-Ras (PDB ID: 3K8Y) for POPs computation.
- Bond-to-bond propensity analysis results: sessionIZNY1I (from https://proteinlens.io/webserver/25897c9d-f41b-4b5a-be97-3d62a948d328/results/383e647b-da1e-488d-a580-86379be090f0)
- Source residue: GNP A528
- Allosteric ligand: Calcium acetate A169, A719

In [ ]:
import POPs
import os
import multiprocessing
import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [ ]:
owd = os.getcwd()
session = 'sessionIZNY1I'

# the residue should be in the following format [('residues number', 'chain')]
source_residues = [('528', 'A')] # this should match the source you used in bond-to-bond propensity analysis
allo_ligs = [('719', 'A'), ('169', 'A')] # this could be an empty list if no ligand is present

pdb_file = f'{session}/{session}_modified.pdb'
bond_file = f'{session}/{session}_graph_bonds.csv'
bond_results_file = f'{session}/{session}_propensity_bonds.csv'

pathway = POPs.PropOptPaths(pdb_file, source_residues, allo_ligs, bond_file, bond_results_file)

# to define starting residues (e.g. active site) and targeting residues (e.g. allosteric site) for POPs
source = pathway.site_format(source_residues)
target = pathway.site_format(allo_ligs)

# pathway.bonded_residues(residue of interest, 
#                         [qs filter to choose bonds with certain qs (set as 0~1 here)], 
#                         [chain filter to ignore certain chain (empty list means all chains are considered])
act_site = pathway.bonded_residues(source, [0, 1], [])
allo_site = pathway.bonded_residues(target, [0, 1], [])

## POPs computation

In [ ]:
# pathway.calculate_POP(prefix given to the file (filename), 
#                       [any chain you want to ignore] e.g. ['B', 'C'], 
#                       act_site_residue, allo_site)
POP_results_folder = f'{owd}/sessionIZNY1I_pathway_analysis/'
os.mkdir(POP_results_folder)
os.chdir(POP_results_folder)
with multiprocessing.Pool(processes = 2) as pool:
    pool.starmap(pathway.calculate_POP, [(session, [], _, allo_site) for _ in act_site])

# the heatmap similar to Figure 1(a) in the paper is saved in the results folder
pathway.plot_heatmap(act_site, allo_site, POP_results_folder)

## Determination of key signalling residues with residue removal

In [ ]:
impt_res_results_dir = f'{owd}/{session}_impt_res'
os.mkdir(impt_res_results_dir)

# res_list contains all natural residues in the protein
res_list = [_.split(' ')[0] + _.split(' ')[1] + ' ' + _.split(' ')[2] for _ in pathway.res_list_no_lig]
# residues other than active and allosteric site residues will be removed
res_to_remove = [i for i in res_list if i not in act_site and i not in allo_site]
chains_to_remove = ['A']
chain_filter = [] # any chain to ignore during the computation
selected_removed_res = []
for i in chains_to_remove:
    selected_removed_res = selected_removed_res + [removed for removed in res_to_remove if removed.endswith(i)]
with multiprocessing.Pool(processes = 2) as pool:
    pool.starmap(pathway.calculate_characteristic_path, 
                 [(session, _, act_site, allo_site, chain_filter, impt_res_results_dir) for _ in selected_removed_res[:5]])